# Step 3: Import Libraries

Import the required libraries:

In [3]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image_dataset_from_directory
import matplotlib.pyplot as plt
import numpy as np


# Step 4: Load the Dataset

You can use a smaller subset of the Cats vs. Dogs dataset from TensorFlow Datasets:

In [1]:
!curl -O https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip
!unzip -q cats_and_dogs_filtered.zip


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 65.4M  100 65.4M    0     0  15.1M      0  0:00:04  0:00:04 --:--:-- 15.1M


# Now, load the training and validation datasets:

# Additional Tip

Make sure that the path 'cats_and_dogs_filtered/train' and 'cats_and_dogs_filtered/validation' is correct and that the dataset has been extracted properly. If the dataset isn't present in the expected directory, you'll need to adjust the path accordingly.

In [4]:
train_ds = image_dataset_from_directory(
    'cats_and_dogs_filtered/train',
    image_size=(180, 180),
    batch_size=32
)

validation_ds = image_dataset_from_directory(
    'cats_and_dogs_filtered/validation',
    image_size=(180, 180),
    batch_size=32
)


Found 2000 files belonging to 2 classes.
Found 1000 files belonging to 2 classes.


# Step 5: Preprocess the Data

Normalize the pixel values:

In [5]:
normalization_layer = tf.keras.layers.Rescaling(1./255)

train_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
validation_ds = validation_ds.map(lambda x, y: (normalization_layer(x), y))


#Step 6: Build the Model

Create a simple CNN model:

In [6]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(180, 180, 3)),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')  # Binary classification
])

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


# Step 7: Train the Model

Train the model on the dataset:

In [7]:
history = model.fit(
    train_ds,
    validation_data=validation_ds,
    epochs=10
)


Epoch 1/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 14s 120ms/step - accuracy: 0.5207 - loss: 0.7796 - val_accuracy: 0.5000 - val_loss: 0.6934
Epoch 2/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 5s 79ms/step - accuracy: 0.5463 - loss: 0.6877 - val_accuracy: 0.5820 - val_loss: 0.6841
Epoch 3/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 3s 55ms/step - accuracy: 0.5868 - loss: 0.6731 - val_accuracy: 0.6390 - val_loss: 0.6697
Epoch 4/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 5s 55ms/step - accuracy: 0.6777 - loss: 0.6174 - val_accuracy: 0.6700 - val_loss: 0.6354
Epoch 5/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 5s 53ms/step - accuracy: 0.7435 - loss: 0.5332 - val_accuracy: 0.6750 - val_loss: 0.6773
Epoch 6/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 3s 52ms/step - accuracy: 0.7957 - loss: 0.4321 - val_accuracy: 0.6210 - val_loss: 0.9191
Epoch 7/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 8s 93ms/step - accuracy: 0.8330 - loss: 0.3559 - val_accuracy: 0.6840 - val_loss: 0.7814
Epoch 8/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 3s 52ms/step - accuracy: 0.9074 - loss: 0.2259 - val_accuracy: 0.6930 -

# Step 8: Evaluate the Model

Check the model's performance:

In [8]:
loss, accuracy = model.evaluate(validation_ds)
print(f'Validation accuracy: {accuracy * 100:.2f}%')


32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 33ms/step - accuracy: 0.6898 - loss: 1.4757
Validation accuracy: 69.90%


# Step 9: Save the Model

Save the trained model so you can use it in Streamlit:

In [9]:
model.save('cats_vs_dogs_model.h5')


In [11]:
with open('app.py', 'w') as f:
    f.write("""
import streamlit as st
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import numpy as np

# Load the model
model = load_model('cats_vs_dogs_model.h5')

st.title('Cats vs. Dogs Classifier')

# Upload an image
uploaded_file = st.file_uploader('Choose an image...', type='jpg')

if uploaded_file is not None:
    img = image.load_img(uploaded_file, target_size=(180, 180))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0) / 255.0  # Normalize

    # Predict
    prediction = model.predict(img_array)
    if prediction[0][0] > 0.5:
        st.write('This is a Dog!')
    else:
        st.write('This is a Cat!')
""")
